In [2]:
from gurobipy import Model, GRB
import pandas as pd
from datetime import datetime

In [21]:
def time_to_minutes(datetime_str):
    # Split the input string to get the date and time parts
    date_part, time_part = datetime_str.split(',')
    
    # Convert the date part to a datetime object
    date_obj = datetime.strptime(date_part.strip(), "%d/%m/%y")
    
    # Get the day of the week (0 = Monday, 6 = Sunday)
    day_of_week = date_obj.weekday()
    
    # Convert the time part to a datetime object
    time_obj = datetime.strptime(time_part.strip(), "%H:%M")
    
    # Calculate the total minutes
    total_minutes = day_of_week * 24 * 60 + time_obj.hour * 60 + time_obj.minute
    return total_minutes

In [3]:
# Read data
fichier = 'instance_WPY_realiste_jalon1.xlsx'
chantiers_df = pd.read_excel(fichier, sheet_name='Chantiers')
machines_df = pd.read_excel(fichier, sheet_name='Machines')
sillons_arrivee_df = pd.read_excel(fichier, sheet_name='Sillons arrivee')
sillons_depart_df = pd.read_excel(fichier, sheet_name='Sillons depart')
correspondances_df = pd.read_excel(fichier, sheet_name='Correspondances')


In [19]:
machines = machines_df['Machine'].tolist()
trains_arr =[]
trains_dep =[]
for x in sillons_arrivee_df['n°TRAIN'].tolist():
    if x not in trains_arr:
        trains_arr.append(x)
for x in sillons_depart_df['n°TRAIN'].tolist():
    if x not in trains_dep:
        trains_dep.append(x)
trains = trains_arr + trains_dep
minutes = list(range(0, 24*60*7))

In [5]:
m = Model('optimisation')

Set parameter Username
Set parameter LicenseID to value 2617662
Academic license - for non-commercial use only - expires 2026-02-03


Variable Binaire $x_{m,h,t}$ m-machine, h-minute, t-train

In [20]:
X = m.addVars(machines, minutes, trains, vtype=GRB.BINARY)

In [ ]:
# Contrainte de disponibilité de la machine pendant les périodes d'indisponibilité
for _, row in machines_df.iterrows():
    task_type = row['Type de tache']
    unavailable_times = row['Indisponibilites']
    
    if unavailable_times != 0:
        periods = unavailable_times.strip('()').split(';')
        for period in periods:
            day_of_week, time_str = period.split(',')
            start_time = time_to_minutes(day_of_week,time_str.split('-')[0])
            end_time = time_to_minutes(day_of_week,time_str.split('-')[1])
            
            
            for h in range(start_time, end_time):
                m.addConstr(sum(x[(m, h, t)] for t in trains for m in machines) == 0)